This notebook receives as input a list of simulation_ids and determines all the champions.

The goal is to determine the feasibility of our simulations compared to the real champions. We expect that the actual champions are the most frequent outcome.

In [1]:
import binarytree as bt
import sqlalchemy
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
engine = sqlalchemy.create_engine('sqlite:///../nba_manager.db')

In [3]:
all_scenarios = pd.read_sql_query('select * from scenarios order by id desc', engine)
latest_scenario = all_scenarios.iloc[0]['scenario_group_id']
print ('latest scenario {}'.format(latest_scenario))

OperationalError: (sqlite3.OperationalError) no such table: scenarios
[SQL: select * from scenarios order by id desc]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
scenarios = pd.read_sql_query('select * from scenarios where scenario_group_id="{}"'.format(latest_scenario), engine)
playoff_brackets = pd.read_sql_table('playoff_brackets', engine)
playoff_brackets.sort_values('id', ascending=False, inplace=True)
teams = pd.read_sql_table('teams', engine)
#simulation_id = playoff_brackets.iloc[0]['simulation_id']
#simulation_year = playoff_brackets.iloc[0]['year']
#nodes_sep_comma = playoff_brackets.iloc[0]['nodes_sep_comma']

In [4]:
simulation_ids = scenarios.simulation_id.unique().tolist()
print (len(simulation_ids))

NameError: name 'scenarios' is not defined

In [7]:
def get_champions(simulation_ids):
    tuples = []
    playoff_brackets = pd.read_sql_table('playoff_brackets', engine)
    playoff_brackets = playoff_brackets[playoff_brackets.simulation_id.isin(simulation_ids)]
    for element in playoff_brackets.itertuples():
        champion_id = element.nodes_sep_comma.split(',')[0]
        tuples.append((element.year, int(champion_id)))
    
    return tuples

In [8]:
#get_champions(simulation_ids)

In [12]:
champion_ids = [i[1] for i in get_champions(simulation_ids)]
champion_df = pd.DataFrame(dict(champion_id=champion_ids))
champion_df_plus = pd.merge(champion_df, teams, left_on='champion_id', right_on='id', copy=True)
champion_df_plus.name.value_counts()[:10]

Brooklyn Nets            28
Los Angeles Clippers     22
Charlotte Hornets        21
Boston Celtics           21
Atlanta Hawks            21
New York Knicks          20
Golden State Warriors    20
San Antonio Spurs        19
Toronto Raptors          19
Memphis Grizzlies        19
Name: name, dtype: int64

### Conclusion #1

In [16]:
vegas_odds = pd.read_clipboard()

In [21]:
vegas_odds['team_name'] = vegas_odds['Team'].apply(lambda x: x.split(',')[0])

In [24]:
vegas_odds.drop(columns=['Team'], inplace=True)

In [25]:
vegas_odds.to_csv('../db/initial_migration/vegas_odds_2017_22.csv', index=False)

In [27]:
vegas_odds['year'] = vegas_odds['Season'].apply(lambda x: x.split('-')[0])

In [29]:
vegas_odds.merge(teams, left_on='team_name', right_on='name')

,Season,Odds,team_name,year,id,name,short_name,conference,division
0,2021-22,1,Brooklyn Nets,2021,2,Brooklyn Nets,BKN,EAST,ATLANTIC
1,2020-21,3,Brooklyn Nets,2020,2,Brooklyn Nets,BKN,EAST,ATLANTIC
2,2019-20,12,Brooklyn Nets,2019,2,Brooklyn Nets,BKN,EAST,ATLANTIC
3,2018-19,28,Brooklyn Nets,2018,2,Brooklyn Nets,BKN,EAST,ATLANTIC
4,2017-18,29,Brooklyn Nets,2017,2,Brooklyn Nets,BKN,EAST,ATLANTIC
5,2021-22,2,Los Angeles Lakers,2021,30,Los Angeles Lakers,LAL,WEST,PACIFIC
6,2020-21,1,Los Angeles Lakers,2020,30,Los Angeles Lakers,LAL,WEST,PACIFIC
7,2019-20,2,Los Angeles Lakers,2019,30,Los Angeles Lakers,LAL,WEST,PACIFIC
8,2018-19,4,Los Angeles Lakers,2018,30,Los Angeles Lakers,LAL,WEST,PACIFIC
9,2017-18,17,Los Angeles Lakers,2017,30,Los Angeles Lakers,LAL,WEST,PACIFIC
